In [41]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import plotly.express as px

# Data Exploration

In [42]:
data_1 = pd.read_csv("/Users/bonniez/Downloads/archive/analyst_ratings_processed.csv", index_col=0)
data_1.head()

,title,date,stock
0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [43]:
data_1['date'] = data_1['date'].str.split(' ', expand=True).iloc[:, 0]

In [44]:
data_1.head()

,title,date,stock
0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A
1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A
2.0,71 Biggest Movers From Friday,2020-05-26,A
3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A
4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A


In [45]:
data_1['stock'].nunique()

6192

In [46]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1400469 entries, 0.0 to 1413848.0
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   title   1400469 non-null  object
 1   date    1399180 non-null  object
 2   stock   1397891 non-null  object
dtypes: object(3)
memory usage: 42.7+ MB


## Drop Null Values

In [47]:
print(data_1.isna().sum())

title       0
date     1289
stock    2578
dtype: int64


In [48]:
print(data_1.shape)

(1400469, 3)


In [49]:
data_1[data_1['date'].isna()]

,title,date,stock
5714.0,Sterne Agee Provides Color on Aaron's,NaN,NaN
7021.0,Goldman Sachs Updates on Car Parts,NaN,NaN
13735.0,American Campus to Offer 15M Shares,NaN,NaN
13764.0,American Campus Communities Reports Q2 FFOM of...,NaN,NaN
14421.0,Positive Seeking Alpha Article on Aceto Corpor...,NaN,NaN
...,...,...,...
1410993.0,Zynga Loses Its CTO of Mobile -AllThingsD,NaN,NaN
1411126.0,Looks Like Zynga Just Bought OMGPOP For $200 M...,NaN,NaN
1411150.0,Zynga Lead Game Designer Leaving the Company -...,NaN,NaN
1412824.0,Waiting on Several IPOs This Morning:,NaN,NaN


In [50]:
data_1.dropna(subset=['date'],inplace=True)
data_1.dropna(subset=['stock'],inplace=True)

In [51]:
print(data_1.shape)

(1397891, 3)


In [52]:
print(data_1.isna().sum())

title    0
date     0
stock    0
dtype: int64


In [53]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1397891 entries, 0.0 to 1413848.0
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   title   1397891 non-null  object
 1   date    1397891 non-null  object
 2   stock   1397891 non-null  object
dtypes: object(3)
memory usage: 42.7+ MB


## Filter stocks

In [54]:
# Filter dataset down to stocks with top 100 number of headlines
top_100_stocks_by_headlines = data_1.groupby('stock').size().reset_index(name='size').sort_values('size', ascending=False).reset_index(drop=True).iloc[:100]
top_100_stocks_by_headlines.head()

,stock,size
0,MRK,3334
1,MS,3242
2,MU,3144
3,NVDA,3133
4,QQQ,3100


In [55]:
data_1 = data_1[data_1['stock'].isin(top_100_stocks_by_headlines.stock)]

In [56]:
data_1.shape

(212110, 3)

## Clean Text Data

In [57]:
data_1['title'] = data_1['title'].str.lower()

In [58]:
data_1.head()

,title,date,stock
0.0,stocks that hit 52-week highs on friday,2020-06-05,A
1.0,stocks that hit 52-week highs on wednesday,2020-06-03,A
2.0,71 biggest movers from friday,2020-05-26,A
3.0,46 stocks moving in friday's mid-day session,2020-05-22,A
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A


### Remove Punctuations

In [59]:
import string

def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])

data_1['title'] = data_1['title'].apply(remove_punctuation)

In [60]:
data_1.head()

,title,date,stock
0.0,stocks that hit week highs on friday,2020-06-05,A
1.0,stocks that hit week highs on wednesday,2020-06-03,A
2.0,biggest movers from friday,2020-05-26,A
3.0,stocks moving in fridays midday session,2020-05-22,A
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A


In [61]:
data_1['tokens'] = data_1['title'].apply(lambda x: x.split())
data_1.head()

,title,date,stock,tokens
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stocks, that, hit, week, highs, on, friday]"
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stocks, that, hit, week, highs, on, wednesday]"
2.0,biggest movers from friday,2020-05-26,A,"[biggest, movers, from, friday]"
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stocks, moving, in, fridays, midday, session]"
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, of, a, securities, maintains, neutral, on,..."


### Remove stop words

In [62]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop = stopwords.words('english')

data_1['tokens'] = data_1['tokens'].apply(lambda x: [word for word in x if word not in stop])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bonniez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
data_1.shape

(212110, 4)

### Lemmatize tokens

In [64]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

data_1['tokens'] = data_1['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


[nltk_data] Downloading package wordnet to /Users/bonniez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [65]:
data_1.head()

,title,date,stock,tokens
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stock, hit, week, high, friday]"
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stock, hit, week, high, wednesday]"
2.0,biggest movers from friday,2020-05-26,A,"[biggest, mover, friday]"
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stock, moving, friday, midday, session]"
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, security, maintains, neutral, agilent, tec..."


In [66]:
data_1['preprocessed_text'] = data_1['tokens'].apply(' '.join)
data_1.head()

,title,date,stock,tokens,preprocessed_text
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stock, hit, week, high, friday]",stock hit week high friday
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stock, hit, week, high, wednesday]",stock hit week high wednesday
2.0,biggest movers from friday,2020-05-26,A,"[biggest, mover, friday]",biggest mover friday
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stock, moving, friday, midday, session]",stock moving friday midday session
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, security, maintains, neutral, agilent, tec...",b security maintains neutral agilent technolog...


In [67]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
# NLTK Sentiment Intensity Analyzer uses a 'Bag of Words' approach
# it removes stop words and scores each word individually before compounding

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/bonniez/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [68]:
data_duplicate = data_1.copy()
data_duplicate['sentiment_score'] = data_duplicate['preprocessed_text'].apply(lambda x: sia.polarity_scores(x)['compound'])
data_duplicate

,title,date,stock,tokens,preprocessed_text,sentiment_score
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stock, hit, week, high, friday]",stock hit week high friday,0.0000
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stock, hit, week, high, wednesday]",stock hit week high wednesday,0.0000
2.0,biggest movers from friday,2020-05-26,A,"[biggest, mover, friday]",biggest mover friday,0.0000
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stock, moving, friday, midday, session]",stock moving friday midday session,0.0000
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, security, maintains, neutral, agilent, tec...",b security maintains neutral agilent technolog...,0.3400
...,...,...,...,...,...,...
1411294.0,is zyngas prethanksgiving ipo a turkey,2011-11-06,ZNGA,"[zyngas, prethanksgiving, ipo, turkey]",zyngas prethanksgiving ipo turkey,0.0000
1411295.0,zynga files new s stament for ipo cash on hand...,2011-11-04,ZNGA,"[zynga, file, new, stament, ipo, cash, hand, s...",zynga file new stament ipo cash hand slightly ...,0.4939
1411296.0,zynga ipo to debut week before thanksgiving ac...,2011-10-24,ZNGA,"[zynga, ipo, debut, week, thanksgiving, accord...",zynga ipo debut week thanksgiving according so...,0.0000
1411297.0,ipo filing zynga amends s filing,2011-10-14,ZNGA,"[ipo, filing, zynga, amends, filing]",ipo filing zynga amends filing,0.0000


In [69]:
data_duplicate['sentiment_score'].describe()

count    212110.000000
mean          0.072493
std           0.291640
min          -0.957800
25%           0.000000
50%           0.000000
75%           0.273200
max           0.974400
Name: sentiment_score, dtype: float64

In [70]:
data_duplicate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212110 entries, 0.0 to 1411298.0
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   title              212110 non-null  object 
 1   date               212110 non-null  object 
 2   stock              212110 non-null  object 
 3   tokens             212110 non-null  object 
 4   preprocessed_text  212110 non-null  object 
 5   sentiment_score    212110 non-null  float64
dtypes: float64(1), object(5)
memory usage: 11.3+ MB


# Pull yfinance data

In [71]:
tickers = data_duplicate.stock.unique()

In [72]:
start_date, end_date = data_duplicate.date.sort_values().iloc[0], data_duplicate.date.sort_values().iloc[-1]

In [73]:
data = yf.download(list(tickers), start=start_date, end=end_date)

[*********************100%%**********************]  100 of 100 completed


13 Failed downloads:
['APC', 'MON', 'ZNGA', 'JCP', 'MYL', 'BBBY', 'AGN']: Exception('%ticker%: No timezone found, symbol may be delisted')
['CHK', 'SHLD']: Exception("%ticker%: Data doesn't exist for startDate = 1240977600, endDate = 1591848000")
['BBRY', 'PCLN', 'GMCR', 'LNKD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2009-04-29 -> 2020-06-11)')


In [74]:
data

Adj Close                                                        \
                    A        ADBE AGN        AIG APC       ATVI        AVGO   
Date                                                                          
2009-04-29  11.629641   26.750000 NaN  18.591230 NaN   9.411404         NaN   
2009-04-30  11.777992   27.350000 NaN  18.195677 NaN   9.446488         NaN   
2009-05-01  12.132750   27.459999 NaN  18.195677 NaN   9.507889         NaN   
2009-05-04  12.526210   26.690001 NaN  19.250492 NaN   9.692080         NaN   
2009-05-05  12.081150   26.889999 NaN  22.810514 NaN   9.621909         NaN   
...               ...         ...  ..        ...  ..        ...         ...   
2020-06-04  89.064148  385.799988 NaN  31.996902 NaN  68.326004  277.644470   
2020-06-05  88.321457  392.899994 NaN  33.041965 NaN  67.954086  285.005951   
2020-06-08  88.233498  397.779999 NaN  35.840893 NaN  69.226456  285.401489   
2020-06-09  87.725334  397.160004 NaN  34.296036 NaN  70.469460  283.424042   
2020-06-10  87.764435  406.820007 NaN  31.951469 NaN  72.250763  283.172363   

                                               ...   Volume           \
                   AXP        AZN        BABA  ...     URBN      VMW   
Date                                           ...                     
2009-04-29   20.127371  10.166079         NaN  ...  9728600  1814000   
2009-04-30   20.345175   9.726616         NaN  ...  9012600  2106300   
2009-05-01   19.594938   9.782243         NaN  ...  6598900  1435300   
2009-05-04   22.007000  10.010320         NaN  ...  5050000  2687900   
2009-05-05   21.434229   9.887938         NaN  ...  5925100  3689100   
...                ...        ...         ...  ...      ...      ...   
2020-06-04  101.979034  49.753910  218.039993  ...  2600700  1318100   
2020-06-05  105.032463  49.735439  219.550003  ...  1548000  1531100   
2020-06-08  108.803795  48.562485  219.000000  ...  1629800  1357900   
2020-06-09  105.731224  48.701019  220.720001  ...  2361900  1193800   
2020-06-10  101.615311  49.541481  223.679993  ...  1656300  1467300   

                                                                          \
                  VZ      WDC        WFC        XLF       XRT       YELP   
Date                                                                       
2009-04-29  21459256  4270400  142750100  192158977  26430400        NaN   
2009-04-30  23924686  7831600  115836200  215537513  29959000        NaN   
2009-05-01  13002285  4485000   96730100  137526705  32675200        NaN   
2009-05-04  16180380  4740400  294065000  260737864  31001000        NaN   
2009-05-05  17482986  4889500  175576000  200012388  17197400        NaN   
...              ...      ...        ...        ...       ...        ...   
2020-06-04  16147300  6973000   61839200   97575100   5712900  1081200.0   
2020-06-05  17105600  7708200  107651200  135859300   3879200  2572200.0   
2020-06-08  14963000  5376700   69051300   80298400   4900800  1194200.0   
2020-06-09  14530800  4124000   51480400   92833100   7829400  1653800.0   
2020-06-10  13649100  4894500   82090000  110848900   5895600   882800.0   

                           
                 YUM ZNGA  
Date                       
2009-04-29   7280216  NaN  
2009-04-30   7283276  NaN  
2009-05-01   6228203  NaN  
2009-05-04  10258764  NaN  
2009-05-05   8831181  NaN  
...              ...  ...  
2020-06-04   2537000  NaN  
2020-06-05   3722800  NaN  
2020-06-08   2748000  NaN  
2020-06-09   2063200  NaN  
2020-06-10   2418600  NaN  

[2799 rows x 600 columns]

In [83]:
# Remove rows with NaN values
data_cleaned = data.dropna(axis=1)
data_cleaned

Adj Close                                                \
                    A        ADBE        AIG       ATVI         AXP   
Date                                                                  
2009-04-29  11.629641   26.750000  18.591230   9.411404   20.127371   
2009-04-30  11.777992   27.350000  18.195677   9.446488   20.345175   
2009-05-01  12.132750   27.459999  18.195677   9.507889   19.594938   
2009-05-04  12.526210   26.690001  19.250492   9.692080   22.007000   
2009-05-05  12.081150   26.889999  22.810514   9.621909   21.434229   
...               ...         ...        ...        ...         ...   
2020-06-04  89.064148  385.799988  31.996902  68.326004  101.979034   
2020-06-05  88.321457  392.899994  33.041965  67.954086  105.032463   
2020-06-08  88.233498  397.779999  35.840893  69.226456  108.803795   
2020-06-09  87.725334  397.160004  34.296036  70.469460  105.731224   
2020-06-10  87.764435  406.820007  31.951469  72.250763  101.615311   

                                                                     ...  \
                  AZN        BAC        BIDU        BIIB        BMY  ...   
Date                                                                 ...   
2009-04-29  10.166079   7.104773   22.729000   47.419998  11.872395  ...   
2009-04-30   9.726616   7.309403   23.290001   48.340000  11.903399  ...   
2009-05-01   9.782243   7.121145   23.561001   46.259998  11.959192  ...   
2009-05-04  10.010320   8.496263   24.825001   46.610001  12.107990  ...   
2009-05-05   9.887938   8.872782   24.980000   46.430000  12.200986  ...   
...               ...        ...         ...         ...        ...  ...   
2020-06-04  49.753910  24.771727  109.059998  299.649994  54.996918  ...   
2020-06-05  49.735439  26.001989  114.400002  306.720001  55.394344  ...   
2020-06-08  48.562485  26.399744  117.029999  301.260010  55.484661  ...   
2020-06-09  48.701019  26.103737  115.830002  293.739990  55.033051  ...   
2020-06-10  49.541481  24.605225  116.809998  295.250000  54.337578  ...   

             Volume                                                            \
                 TM       TXN     URBN      VMW        VZ      WDC        WFC   
Date                                                                            
2009-04-29  1034200  11803800  9728600  1814000  21459256  4270400  142750100   
2009-04-30  1408500  18908600  9012600  2106300  23924686  7831600  115836200   
2009-05-01   875600  16325400  6598900  1435300  13002285  4485000   96730100   
2009-05-04   936100  16309000  5050000  2687900  16180380  4740400  294065000   
2009-05-05   780600  17431400  5925100  3689100  17482986  4889500  175576000   
...             ...       ...      ...      ...       ...      ...        ...   
2020-06-04   196900   5052500  2600700  1318100  16147300  6973000   61839200   
2020-06-05   238900   7788500  1548000  1531100  17105600  7708200  107651200   
2020-06-08   189500   5602500  1629800  1357900  14963000  5376700   69051300   
2020-06-09   195800   4450000  2361900  1193800  14530800  4124000   51480400   
2020-06-10   158400   4005700  1656300  1467300  13649100  4894500   82090000   

                                           
                  XLF       XRT       YUM  
Date                                       
2009-04-29  192158977  26430400   7280216  
2009-04-30  215537513  29959000   7283276  
2009-05-01  137526705  32675200   6228203  
2009-05-04  260737864  31001000  10258764  
2009-05-05  200012388  17197400   8831181  
...               ...       ...       ...  
2020-06-04   97575100   5712900   2537000  
2020-06-05  135859300   3879200   3722800  
2020-06-08   80298400   4900800   2748000  
2020-06-09   92833100   7829400   2063200  
2020-06-10  110848900   5895600   2418600  

[2799 rows x 462 columns]